<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer:
- conatins all the neurons that accept input for processing (1 neuron per feature)
- processes inputs into a format the hidden layers can use

### Hidden Layer:
- can be more than 1
- transforms information from the input layer into a format the ouput layer can use
- 'brain' of the nerual network
- all the math-y 'secret sauce' happens here
- maps inputs to outputs

### Output Layer:
- accepts processed data from hidden layers, scales/transforms/converts it into desired output format

### Neuron:
- single node that receives input and (potentially) passes processed data on as output
- processes data with the 'weight'

### Weight:
- similar to a coefficient in linear regression
- signal?? applied to the data by a node
- node modifies input by weight then (potentially) passes to output

### Activation Function:
- process by which a node transforms its input before spitting out the data
- decides if a node 'fires' (outputs or not)
- usually the same for all nodes in a layer
- sigmoid, tanh, step, relu

### Node Map:
- visual aid of the layout of a neural network
- flowchart
- overview of the neural network connections

### Perceptron:
- simplest neural network / single node of a neural network
- takes any number of inputs, processes it, outputs something
- activation_function(sum(input(s) * weight)) -> output if threshold is reached

## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

inputs -> input_layer -> hidden_layer(s) -> output_layer -> outputs

input_layer
- receive inputs -> transform into usable form -> hidden_layer(s)

hidden_layer
- receive inputs -> apply weight -> sum weights -> add bias -> activation function -> if above threshold -> output
- repeat through all hidden layers -> output_layer

output_layer
- receive inputs -> transform into desired output -> output

## Write your own perceptron code that can correctly classify (99.0% accuracy) a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [341]:
import pandas as pd
data = { 'x1': [0,1,0,1],
         'x2': [0,0,1,1],
         'y':  [1,1,1,0]
       }

df = pd.DataFrame.from_dict(data).astype('int')
df

,x1,x2,y
0,0,0,1
1,1,0,1
2,0,1,1
3,1,1,0


In [344]:
import numpy as np

target = 'y'
features = ['x1', 'x2']

X_train = df[features]
y_train = df[target]

**Sigmoid activation function and its derivative for updating weights**

In [343]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx * (1 - sx)

In [ ]:
# add bias to training data
X_train['bias'] = 1

In [345]:
# Initialize random weights for each of our inputs
weights = 2 * np.random.random((X_train.shape[1],1)) - 1
print('weights before training')
print(weights)
print()

# each iteration is an epoch
for iteration in range(1000):
    # weighted sum
    weighted_sum = np.dot(X_train, weights)
    
    # activate
    activated_output = sigmoid(weighted_sum + bias)
    
    # calculate error
    error = y_train.values.reshape(-1, 1) - activated_output
    
    # calculate adjustments
    adjustments = error * sigmoid_derivative(activated_output)
    
    # update the weights
    weights += np.dot(X_train.T, adjustments)
    
print('weights after training')
print(weights)
print()
print('outputs after training')
print(activated_output)
print()


weights before training
[[0.38006156]
 [0.46706731]]

weights after training
[[-3.39775202]
 [-3.39775202]]

outputs after training
[[0.99330715]
 [0.83233234]
 [0.83233234]
 [0.14239924]]



In [346]:
from sklearn.metrics import accuracy_score

y_pred = np.round(activated_output).astype(int)
print('predictions:')
print(y_pred)
print()
print('accuracy:', accuracy_score(y_train, y_pred))

predictions:
[[1]
 [1]
 [1]
 [0]]

accuracy: 1.0


## Implement your own Perceptron Class and use it to classify a binary dataset: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [239]:
diabetes = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [240]:
diabetes.shape

(768, 9)

In [241]:
diabetes.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

Although neural networks can handle non-normalized data, scaling or normalizing your data will improve your neural network's learning speed. Try to apply the sklearn `MinMaxScaler` or `Normalizer` to your diabetes dataset. 

In [242]:
target = 'Outcome'
features = diabetes.drop(columns=target).columns

X = diabetes[features]
y = diabetes[target]

In [329]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels

class Perceptron(BaseEstimator, ClassifierMixin):
    def __init__(self, n_iter=100):
        self.n_iter = n_iter
    
    def __add_bias(self, x):
        return np.append(x, np.ones((x.shape[0], 1)), 1)
    
    def __sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def __sigmoid_derivative(self, x):
        sx = self.__sigmoid(x)
        return sx * (1 - sx)

    def fit(self, X, y):
        X, y = check_X_y(X, y)
        self.classes_ = unique_labels(y)
        self.X_ = self.__add_bias(X)
        self.y_ = y
        
        weights = 2 * np.random.random((self.X_.shape[1], 1)) - 1
        
        for i in range(self.n_iter):
            weighted_sum = np.dot(self.X_, weights)

            # Activate!
            activated_output = self.__sigmoid(weighted_sum)

            # Cac error
            error = y.reshape(-1, 1) - activated_output
            adjustments = error * self.__sigmoid_derivative(activated_output)

            # Update the Weights
            weights += np.dot(self.X_.T, adjustments)
        
        self.weights_ = weights
        
        return self
    
    def predict(self, X):
        check_is_fitted(self)
        X = check_array(X)
        X = self.__add_bias(X)
        
        weighted_sum = np.dot(X, self.weights_)
        pred_proba = self.__sigmoid(weighted_sum)
        
        pred = np.round(pred_proba).astype(int)
        
        return pred.reshape(1, -1)

    def predict_proba(self, X):
        check_is_fitted(self)
        X = check_array(X)
        X = self.__add_bias(X)
        
        weighted_sum = np.dot(X, self.weights_)
        pred_proba = self.__sigmoid(weighted_sum)
                
        return pred_proba.reshape(1, -1)

In [340]:
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
#     ('norm', Normalizer()),
    ('scale', StandardScaler()),
    ('clf', Perceptron(n_iter=7000))
])

y_pred = pipeline.fit(X, y).predict(X)

accuracy_score(y, y_pred[0])

0.7096354166666666

## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?